In [2]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('perfume.db')
cursor = conn.cursor()

# Step 1: Drop the column main_accords_dirty
drop_column_query = "ALTER TABLE No_Reviews DROP COLUMN main_accords_dirty"
cursor.execute(drop_column_query)

# Step 2: Split the data in the Maine_accords column into separate accord strings
select_query = "SELECT Maine_accords FROM No_Reviews"
cursor.execute(select_query)
rows = cursor.fetchall()

# Step 3: Concatenate the accord strings for each entry with commas
updated_rows = []
for row in rows:
    main_accords = row[0]
    if main_accords is not None:
        main_accords = main_accords.strip('[]').replace('"', '').split(', ')
        accords_str = ', '.join([f'accord {i+1}' for i in range(len(main_accords))])
    else:
        accords_str = None
    updated_rows.append((accords_str,))

# Step 4: Create a new column to store the concatenated accord strings
create_column_query = "ALTER TABLE No_Reviews ADD COLUMN all_accords TEXT"
cursor.execute(create_column_query)

# Step 5: Update the table with the concatenated accord strings
update_query = "UPDATE No_Reviews SET all_accords = ? WHERE rowid = ?"
cursor.executemany(update_query, updated_rows)

# Step 6: Commit the changes and close the connection
conn.commit()
conn.close()

OperationalError: no such column: "main_accords_dirty"